<center><img src="../image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 8.机器人球体追踪

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：8.机器人球体追踪.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：机器人球体追踪<br>

In [1]:
import cv2
import numpy as np
from LOBOROBOT import LOBOROBOT # 载入机器人库
import  RPi.GPIO as GPIO
import time  

In [2]:
kernel = np.ones((5,5),np.uint8)
# 按键及LED初始化
BtnPin  = 19
Gpin    = 5
Rpin    = 6

In [3]:
clbrobot = LOBOROBOT()  # 实例化机器人对象

# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,90)  # 底座舵机 90 
clbrobot.set_servo_angle(9,30)  # 顶部舵机 30
time.sleep(0.5)

## 打开摄像头

In [4]:
cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
cam.set(3,320)
cam.set(4,240)
cam.isOpened()

True

In [5]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

FGmaskComp_img = widgets.Image(format='jpeg', width=320, height=240)
frame_img = widgets.Image(format='jpeg', width=320, height=240)

dispaly_img = widgets.HBox([FGmaskComp_img,frame_img])
display(dispaly_img)

In [6]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [7]:
hueLower = widgets.IntSlider(min=12,max=179,step=1,description='hueLower：',value=20)
hueUpper = widgets.IntSlider(min=30,max=179,step=1,description='hueUpper：',value=30)

satLow = widgets.IntSlider(min=50,max=255,step=1,description='satLow：',value=50)
satHigh = widgets.IntSlider(min=255,max=255,step=1,description='satHigh：',value=255)

valLow = widgets.IntSlider(min=50,max=255,step=1,description='valLow：',value=50)
valHigh = widgets.IntSlider(min=255,max=255,step=1,description='valHigh：',value=255)

slider_img=widgets.VBox([
              widgets.HBox([hueLower,hueUpper]),
              widgets.HBox([satLow,satHigh]),
              widgets.HBox([valLow,valHigh])
             ])
display(slider_img)

## 按键处理函数

In [8]:
def keysacn():
    val = GPIO.input(BtnPin)
    while GPIO.input(BtnPin) == False:
        val = GPIO.input(BtnPin)
    while GPIO.input(BtnPin) == True:
        time.sleep(0.01)
        val = GPIO.input(BtnPin)
        if val == True:
            GPIO.output(Rpin,1)
            while GPIO.input(BtnPin) == False:
                GPIO.output(Rpin,0)
        else:
            GPIO.output(Rpin,0)

In [9]:
GPIO.setwarnings(False) 
GPIO.setmode(GPIO.BCM)
GPIO.setup(Gpin, GPIO.OUT)     # 设置绿色Led引脚模式输出
GPIO.setup(Rpin, GPIO.OUT)     # 设置红色Led引脚模式输出
GPIO.setup(BtnPin, GPIO.IN, pull_up_down=GPIO.PUD_UP)    # 设置输入BtnPin模式，拉高至高电平(3.3V) 

In [10]:
def Video_display():
    # 键盘控制函数 
    keysacn()
    while(1):
        buzz = 0
        _, frame = cam.read()
        frame=cv2.flip(frame, 1) 
        
        # 转换到HSV
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        hue,sat,val = cv2.split(hsv)
        
        hueLow=hueLower.value
        hueUp=hueUpper.value
    
        Ls= satLow.value
        Us = satHigh.value
    
        Lv=valLow.value
        Uv=valHigh.value
    
        l_b=np.array([hueLow,Ls,Lv])
        u_b=np.array([hueUp,Us,Uv])
    
    
        FGmaskComp=cv2.inRange(hsv,l_b,u_b)
        FGmaskComp_img.value = bgr8_to_jpeg(FGmaskComp)
        
        # 应用阈值
        hthresh = cv2.inRange(np.array(hue),np.array(hueLow),np.array(hueUp))
        sthresh = cv2.inRange(np.array(sat),np.array(Ls),np.array(Us))
        vthresh = cv2.inRange(np.array(val),np.array(Lv),np.array(Uv))
    
        # h s和v
        tracking = cv2.bitwise_and(hthresh,cv2.bitwise_and(sthresh,vthresh))
    
        # 一些morpholigical过滤
        dilation = cv2.dilate(tracking,kernel,iterations = 1)
        closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel)
        closing = cv2.GaussianBlur(closing,(5,5),0)
    
        # 使用HoughCircles检测圆
        circles = cv2.HoughCircles(closing,cv2.HOUGH_GRADIENT,2,120,param1=120,param2=50,minRadius=10,maxRadius=0)
        # circles = np.uint16(np.around(circles))
        clbrobot.t_stop(0)
    
        # 画圆圈
        if circles is not None:
            x, y, r = circles[0][0]
            x_p = int(round(x))
            print(x_p)
            for i in circles[0,:]:
                    # 如果球很远，用绿色画出来
                    if int(round(i[2])) < 30:
                            cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),int(round(i[2])),(0,255,0),5)
                            cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),2,(0,255,0),10)
                    # 或者用红色画
                    elif int(round(i[2])) > 35:
                            cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),int(round(i[2])),(0,0,255),5)
                            cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),2,(0,0,255),10)
                            buzz = 1  
            # 设定一个范围
            x_Lower = 150
            x_Upper = 180
            # 判断X方向范围来判断机器人的运动
            if x_p > x_Lower and x_p < x_Upper:
                    clbrobot.t_up(30,0)
            elif x_p < x_Lower:
                clbrobot.turnLeft(20,0)              
            elif x_p > x_Upper:
                clbrobot.turnRight(20,0)    
            else:
                clbrobot.t_stop(0)
        # 在帧中显示结果
        frame_img.value = bgr8_to_jpeg(frame)

In [11]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

123
155
69
119
117
143
157
235
201
121
53
139
37
15
223
217
201
105
179
277
231
57
135
89
83
49
319
319
203
189
187
191
189
189
191
183
167
145
123
115
91
63
41
49
67
97
119
137
149
145
163
187
213
237
255
269
243
221
197
199
197
201
191
187
173
145
115
87
75
291
197
163
155
123
249
271
167
181
265
263
209
143
133
133
169
173
171
169
177
175
173
179
187
181
183
175
159
159
259
209
213
215
223
233
225
227
207
183
169
141
113
99
87
65
51
21
35
51
67
97
109
123
151
175
203
215
233
257
241
219
193
187
193
195
201
179
145
135
123
89
63
95
111
97
105
101
115
135
223
227
241
245
249
235
213
157
151
175
179
183
189
189
193
195
177
171
151
133
121
77
3
49
49
75
63
117
137
121
117
45
99
131
161
219
221
221
239
237
221
213
181
175
167
165
151
137
107
95
63
41
39
23
31
53
67
79
101
107
115
127
147
173
199
223
239
269
295
259
243
213
227
233
227
235
209
201
171
143
113
107
81
219
257
221
207
209
213
213
217
219
203
185
165
135
115
97
75
51
39
49
69
83
109
121
143
153
167
191
213
239
261
267
275
255

In [14]:
# 结束线程
clbrobot.t_stop(0) # 停止电机
stop_thread(t)

ValueError: invalid thread id